In [4]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers import BertModel
os.environ["CUDA_VISIBLE_DEVICES"]='0'
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
MODEL_NAME='cl-tohoku/bert-base-japanese-whole-word-masking'
batch=1

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [7]:
df_one_month = pd.read_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/2022101month_per_hour_noGeo.csv',engine='python')

In [9]:
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
#code_estimate_model_path = '/home/is/shuntaro-o/SharedTask_main/地理コード/Tokyo/model/epoch=2-step=2400000.ckpt'＃

class BertForSequenceClassifier_pl(pl.LightningModule):
    def __init__(self, model_name, lr, num_class):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters()

        # BERTのロード
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_class)
        self.criterion = nn.CrossEntropyLoss()

        # BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        preds= self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        #print(f"tihi is {loss}")
        return loss, preds

    # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        self.log('train_loss', loss)
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # epoch終了時にvalidationのlossとaccuracyを記録
    def validation_epoch_end(self, outputs, mode="val"):
        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"{mode}_loss", epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)


    # testデータのlossとaccuracyを算出（validationの使いまわし）
    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, "test")

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [11]:
code_estimate_model_path_japan = "/home/is/shuntaro-o/dev/SharedTask_FlaskApp/Flask/app/models/japan_model.ckpt"

In [12]:
arg2mesh_japan = [
            3927,
            3928,
            3933,
            3942,
            3945,
            4027,
            4028,
            4033,
            4037,
            4040,
            4042,
            4043,
            4128,
            4129,
            4130,
            4133,
            4134,
            4135,
            4140,
            4142,
            4143,
            4228,
            4229,
            4231,
            4233,
            4236,
            4243,
            4330,
            4331,
            4332,
            4340,
            4428,
            4432,
            4442,
            4530,
            4531,
            4532,
            4539,
            4540,
            4541,
            4542,
            4628,
            4630,
            4631,
            4632,
            4639,
            4640,
            4641,
            4729,
            4730,
            4731,
            4732,
            4739,
            4828,
            4829,
            4830,
            4831,
            4832,
            4837,
            4838,
            4841,
            4842,
            4844,
            4845,
            4928,
            4929,
            4930,
            4931,
            4932,
            4933,
            4934,
            4937,
            4938,
            4939,
            4940,
            5029,
            5030,
            5031,
            5032,
            5033,
            5034,
            5035,
            5036,
            5037,
            5039,
            5040,
            5041,
            5042,
            5128,
            5129,
            5130,
            5131,
            5132,
            5133,
            5134,
            5135,
            5136,
            5137,
            5138,
            5139,
            5140,
            5141,
            5142,
            5144,
            5227,
            5228,
            5229,
            5231,
            5232,
            5233,
            5234,
            5235,
            5236,
            5237,
            5238,
            5239,
            5240,
            5241,
            5242,
            5243,
            5244,
            5332,
            5333,
            5334,
            5335,
            5336,
            5337,
            5338,
            5339,
            5340,
            5341,
            5342,
            5434,
            5436,
            5437,
            5438,
            5439,
            5440,
            5441,
            5442,
            5444,
            5445,
            5536,
            5537,
            5538,
            5539,
            5540,
            5541,
            5542,
            5543,
            5544,
            5545,
            5630,
            5636,
            5637,
            5638,
            5639,
            5640,
            5641,
            5642,
            5643,
            5644,
            5645,
            5736,
            5738,
            5739,
            5740,
            5741,
            5742,
            5743,
            5744,
            5745,
            5839,
            5840,
            5841,
            5842,
            5843,
            5844,
            5935,
            5937,
            5939,
            5940,
            5941,
            5942,
            5943,
            5944,
            5945,
            6038,
            6039,
            6040,
            6041,
            6042,
            6043,
            6044,
            6139,
            6140,
            6141,
            6142,
            6143,
            6239,
            6240,
            6241,
            6242,
            6243,
            6244,
            6339,
            6340,
            6341,
            6342,
            6343,
            6344,
            6345,
            6432,
            6437,
            6439,
            6440,
            6441,
            6442,
            6443,
            6444,
            6445,
            6446,
            6537,
            6540,
            6541,
            6542,
            6543,
            6544,
            6545,
            6636,
            6640,
            6641,
            6642,
            6643,
            6644,
            6645,
            6741,
            6742,
            6743,
            6745,
            6830,
            6831,
            6840,
            6841,
            6843,
        ]
df_one_month["text"]=df_one_month["text"].astype(str)
sentences_text_test=df_one_month.text.values
i=0
estimation=[]
model = BertForSequenceClassifier_pl.load_from_checkpoint(code_estimate_model_path_japan)
bert=model.bert.cuda()
classifier=model.classifier.cuda()
for sentence in sentences_text_test:
    text=sentence
    encoding = tokenizer(
    text,
    max_length = 107,           # 文章の長さを固定（Padding/Trancatinating）
    pad_to_max_length = True,# PADDINGで埋める
    truncation=True,
    padding = 'longest',
    return_tensors='pt')
    encoding = { k: v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert(**encoding)
        ans=classifier(output.pooler_output)
        ans = ans.to('cpu').detach().numpy().copy()
        ans=np.argmax(ans)
        convert_ans = arg2mesh_japan[ans]
        estimation.append(convert_ans)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
estimation=pd.Series(estimation)
df_one_month=pd.concat([df_one_month,estimation],axis=1)
df_one_month=df_one_month.rename(columns={0: 'code_estimation'})

In [ ]:
df_one_month.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/202210noGeo_ad_estimate.csv')